In [1]:
import sys
import os
from pathlib import Path

# Get project root
ROOT = Path(os.path.abspath('')).parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(ROOT / '.env')

# Import our modules
from src.db import get_connection
from src.constants import env
from src.image import ImageProcessor

In [2]:
# Initialize Oracle connection and test
with get_connection() as conn:
    cur = conn.cursor()
    cur.execute('SELECT 1 FROM dual')
    result = cur.fetchone()
    print(f'Oracle connection test successful: {result[0]}')

Oracle connection test successful: 1


In [3]:
# Check messages in database
with get_connection() as conn:
    cur = conn.cursor()
    cur.execute("""
        SELECT COUNT(*) as message_count,
               MIN(message_ts) as first_msg,
               MAX(message_ts) as last_msg
        FROM telegram_raw.messages
    """)
    count, first, last = cur.fetchone()
    print(f'\nMessages in database:')
    print(f'Total messages: {count}')
    print(f'First message: {first}')
    print(f'Last message: {last}')

Connection failed: ORA-00942: table or view "TELEGRAM_RAW"."MESSAGES" does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/


DatabaseError: ORA-00942: table or view "TELEGRAM_RAW"."MESSAGES" does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/

In [ ]:
# Get a sample message with photo
with get_connection() as conn:
    cur = conn.cursor()
    cur.execute("""
        SELECT message_id, payload
        FROM telegram_raw.messages
        WHERE JSON_EXISTS(payload, '$.photo')
        ORDER BY message_ts DESC
        FETCH FIRST 1 ROW ONLY
    """)
    
    if cur.rowcount == 0:
        print('No messages with photos found!')
        raise SystemExit
    
    message_id, payload = cur.fetchone()
    message = json.loads(payload)
    
    print(f'\nSample message with photo:')
    print(f'Message ID: {message_id}')
    print(f'Channel: {message.get("channel_slug")}')
    print(f'Date: {message.get("date")}')
    print(f'Photo info: {message.get("photo")}')

In [ ]:
pip install -r ../requirements.txt